In [1]:
import pandas as pd
import numpy as np

from scipy import integrate

import warnings
warnings.filterwarnings('ignore')

In [2]:
## helper functions

from scipy.stats import norm

def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

def Black76Lognormal(F, K, r, sigma, T, opt):
    d1 = (np.log(F/K)+(sigma*sigma/2)*T)/(sigma*np.sqrt(T))
    d2 = d1-sigma*np.sqrt(T)
    if opt == 'Call':
        return F*np.exp(-r*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    elif opt == 'Put':
        return K*np.exp(-r*T)*norm.cdf(-d2) - F*np.exp(-r*T)*norm.cdf(-d1)

$$CMS\;Rate\;Payoff =  D(0,T) F + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK $$

### IRR
From cms_replication.ipynb

In [3]:
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative

### CMS Rate Payment $g(K)$

\begin{align*}

g(K) &= CMS 10Y ^{\frac{1}{4}} - 0.04^{\frac{1}{2}} \\
&= K^{\frac{1}{4}} - 0.2 \\ \\
g'(K) &= \frac{1}{4} K^{-\frac{3}{4}} \\ \\
g''(K) &= -\frac{3}{16} K^{-\frac{7}{4}}

\end{align*}

$$ 


In [4]:
# template from cms_replication.ipynb
# altered to fit the payoff for question

def g_0(K):
    return K**(1/4) - 0.2

def g_1(K):
    return (1/4)*K**(-3/4)

def g_2(K): 
    return (-3/16) * K ** (-7/4)

### $h(K)$

In [5]:
# from cms_replication.ipynb

def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative

\begin{align*}
V^{pay}(K) &= Black76Call(S_{n,N}(0),K,\sigma_{n,N},T) \\
V^{rec}(K) &= Black76Put(S_{n,N}(0),K,\sigma_{n,N},T)
\end{align*}

where $S_{n,N}(0)=F$ is today's $(T=5)$ forward swap rate $(5 X 10)$ calculated based on the curves we bootstrapped, and $\sigma_{n,N}$ is the SABR implied volatility calibrated to swaption market data.

In [6]:
### Getting Data

oisDiscFactors = pd.read_excel("Data_1_Output.xlsx", sheet_name='oisDiscFactors')
fwdSwapRates = pd.read_excel("Data_1_Output.xlsx", sheet_name='fwdSwapRates')

In [7]:
oisDiscFactors

,T0,TN,DiscountFactor
0,0,0.0,1.000000
1,0,0.5,0.998752
2,0,1.0,0.997009
3,0,1.5,0.995270
4,0,2.0,0.993531
...,...,...,...
56,0,28.0,0.857703
57,0,28.5,0.855126
58,0,29.0,0.852548
59,0,29.5,0.849986


In [8]:
fwdSwapRates

,Start,Tenor,liborForwardSwapRate
0,1Y,1Y,0.032007
1,1Y,2Y,0.033259
2,1Y,3Y,0.034011
3,1Y,5Y,0.035255
4,1Y,10Y,0.038428
5,5Y,1Y,0.039274
6,5Y,2Y,0.040075
7,5Y,3Y,0.040072
8,5Y,5Y,0.041093
9,5Y,10Y,0.043634


In [9]:
CMS_10x5 = pd.read_excel('Data_3_Output.xlsx', sheet_name = 'CMS 10x5')
CMS_10x5.drop(columns = 'Unnamed: 0', inplace=True)
CMS_10x5

,Expiry,Tenor,ForwardRate,Alpha,Rho,Nu,CMSRate
0,0.5,10,0.044361,0.171109,-0.264508,0.777964,0.044663
1,1.0,10,0.045554,0.171109,-0.264508,0.777964,0.046200
2,1.5,10,0.046388,0.171285,-0.280654,0.746384,0.047419
3,2.0,10,0.047239,0.171462,-0.296801,0.714804,0.048701
4,2.5,10,0.048128,0.171638,-0.312948,0.683224,0.050067
5,3.0,10,0.048953,0.171815,-0.329095,0.651644,0.051402
6,3.5,10,0.049816,0.171992,-0.345242,0.620064,0.052813
7,4.0,10,0.050738,0.172168,-0.361388,0.588484,0.054319
8,4.5,10,0.051704,0.172345,-0.377535,0.556904,0.055903
9,5.0,10,0.052852,0.172521,-0.393682,0.525324,0.057725


### Question 1

$$CMS = D(0,T) F + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK$$

In [10]:
# 5 X 10 Forward Swap Rate
F = CMS_10x5.iloc[9, 6]

# D_ois(0, 5) 
D = oisDiscFactors.loc[oisDiscFactors['TN'] == 5.0]['DiscountFactor'].values[0]

# parameters from SABR calibration 
alpha = CMS_10x5.iloc[9, 3]
beta = 0.9
rho = CMS_10x5.iloc[9, 4]
nu = CMS_10x5.iloc[9, 5]

# CMS parameters

T = 5 
tenor = 10
paymentFreq = 2 # semi-annual


term1 = D * g_0(F) 

term2 = integrate.quad(lambda x: h_2(x, paymentFreq, tenor)*Black76Lognormal(F, x, 0, SABR(F, x, T, alpha, 0.9, rho, nu),T, "Put")
                       ,0, F) 
term3 = integrate.quad(lambda x: h_2(x, paymentFreq, tenor)*Black76Lognormal(F, x, 0, SABR(F, x, T, alpha, 0.9, rho, nu),T, "Call")
                       ,0, F) 

PVoption = term1 + term2[0] + term3[0]

print(f'PV of Option: {PVoption}')

PV of Option: 0.34307451771723385


### Question 2


From notes, CMS Caplet reduces to: 
$$ CMS\;Caplet = V^{pay}(L)h'(L) + \int_{L}^{\infty} h''(K) V^{pay}(K) dK$$ 
$$ where \;\; F \;> (0.04^{\frac{1}{2}})^{4} = L$$

In [11]:
# 5 X 10 Forward Swap Rate
F = CMS_10x5.iloc[9, 6]

# D_ois(0, 5) 
D = oisDiscFactors.loc[oisDiscFactors['TN'] == 5.0]['DiscountFactor'].values[0]

# parameters from SABR calibration 
alpha = CMS_10x5.iloc[9, 3]
beta = 0.9
rho = CMS_10x5.iloc[9, 4]
nu = CMS_10x5.iloc[9, 5]


# CMS parameters

T = 5 
tenor = 10
paymentFreq = 2 # semi-annual
L = (0.04**0.5) ** 4 


### caplet has different h(K), s5 notes pg20

def caplet_h1(K, L, m, N):
    return (IRR_0(K, m, N) - (IRR_1(K, m, N)*(K-L))) / (IRR_0(K, m, N)**2)

def caplet_h2(K, L, m, N):
    term1 = ((-IRR_2(K, m, N)*(K-L)) - (2*IRR_1(K, m , N))) / (IRR_0(K,m,N)**2)
    term2 = (2*(IRR_1(K,m,N)**2)*(K-L)) / (IRR_0(K,m,N)**3)
    return term1 + term2 



capletTerm1 = caplet_h1(F, L, paymentFreq, tenor) * Black76Lognormal(F, L, 0, SABR(F, L, T, alpha, 0.9, rho, nu),T,"Call")
capletTerm2 = integrate.quad(lambda x: caplet_h2(x, L, paymentFreq, tenor)*Black76Lognormal(F, x, 0, SABR(F, x, T, alpha, 0.9, rho, nu),T,"Call")
                       ,L ,10000000)

PVcaplet = capletTerm1 + capletTerm2[0]

print(f'PV of Caplet: {PVcaplet}')


PV of Caplet: 100.7514703763541
